# 🔢 Edge-Magic Total Labeling (EMTL) Tutorial

**A Comprehensive Guide to Graph Labeling with Constraint Programming**

---

This notebook provides an interactive tutorial on Edge-Magic Total Labelings,
covering both the mathematical theory and practical implementation.

## Table of Contents

1. Introduction to Graph Labeling
2. Mathematical Definition
3. The Graph Structure
4. Using the EMTL Solver
5. Visualizing Results
6. Exploring Parameters
7. Exercises


## Setup

First, let's import the necessary modules and set up our environment.


In [ ]:
# Add parent directory to path
import sys
sys.path.insert(0, '..')

# Import the EMTL solver components
from emtl_solver import (
    solve_emtl,
    GraphParameters,
    GraphConstructor,
    EMTLSolver,
    EMTLVisualizer,
    EMTLResult,
    SolverStatus,
)

# Standard imports
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# Configure matplotlib for notebook display
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

print("✓ All imports successful!")


## 1. Introduction to Graph Labeling

**Graph labeling** is a branch of graph theory where we assign labels (usually integers) to vertices and/or edges of a graph subject to certain constraints.

### Why Study Graph Labelings?

- **Theoretical Interest**: Beautiful mathematical structures and open problems
- **Applications**: Network design, scheduling, cryptography, coding theory
- **Computational Challenges**: Many labeling problems are NP-complete

### Types of Graph Labelings

| Type | What's Labeled | Example Constraint |
|------|----------------|-------------------|
| Graceful | Vertices | Edge labels = vertex difference |
| Harmonious | Vertices | Edge sums are distinct |
| **Edge-Magic Total** | Both | **Edge sums are constant** |
| Antimagic | Edges | Edge sums are distinct |


## 2. Mathematical Definition

### Edge-Magic Total Labeling (EMTL)

Let G = (V, E) be a graph with p vertices and q edges.

An **Edge-Magic Total Labeling** is a bijection:

**f: V ∪ E → {1, 2, 3, ..., p + q}**

such that for **every** edge uv ∈ E:

**f(u) + f(uv) + f(v) = k**

where k is a constant called the **magic constant** or **magic sum**.

### Key Properties

1. **Bijection**: Each number 1 to p+q is used exactly once
2. **Constant Sum**: Every edge "weighs" the same (vertex labels + edge label)
3. **Magic Constant**: The value k is unique for a given labeling


## 3. The Graph Structure

Our EMTL solver works with a specific family of graphs defined by parameters (m, n, k, t).

### Vertex Partitions

The vertex set V is divided into four disjoint sets:

| Set | Size | Description |
|-----|------|-------------|
| A | m | First partition |
| B | n | Second partition |
| C | n | Third partition (same size as B) |
| D | k | Fourth partition |

**Total vertices:** |V| = m + 2n + k

### Edge Structure

| Between | Type | Edges |
|---------|------|-------|
| A and B | Complete bipartite K_{m,n} | m × n |
| B and C | t-regular bipartite | n × t |
| C and D | Complete bipartite K_{n,k} | n × k |

**Total edges:** |E| = mn + nt + nk


In [ ]:
# Let's explore the graph structure with concrete parameters

# Define parameters
m, n, k, t = 2, 3, 2, 2

# Create parameters object
params = GraphParameters(m=m, n=n, k=k, t=t)

print(f"Graph Parameters: {params}")
print("═" * 50)

print(f"\nVertex Counts:")
print(f"  |A| = {m}")
print(f"  |B| = {n}")
print(f"  |C| = {n}")
print(f"  |D| = {k}")
print(f"  Total |V| = {params.num_vertices}")

print(f"\nEdge Counts:")
print(f"  A-B edges (complete): {m} × {n} = {m*n}")
print(f"  B-C edges (t-regular): {n} × {t} = {n*t}")
print(f"  C-D edges (complete): {n} × {k} = {n*k}")
print(f"  Total |E| = {params.num_edges}")

print(f"\nLabels needed: {params.total_labels}")


## 4. Using the EMTL Solver

Now let's use our solver to find an Edge-Magic Total Labeling!


In [ ]:
# Solve for an EMTL
result = solve_emtl(
    m=2, n=2, k=2, t=1,
    timeout=60,
    visualize=False,  # We'll visualize manually
    verbose=True
)


In [ ]:
# Check the result
if result.exists:
    print("🎉 EMTL Found!")
    print(f"Magic constant: k = {result.magic_constant}")
    print(f"Solve time: {result.solve_time:.3f} seconds")
    
    print("\n" + "═" * 60)
    print("VERTEX LABELS:")
    for partition in ['A', 'B', 'C', 'D']:
        print(f"\n  Set {partition}:")
        for v in result.vertex_sets[partition]:
            print(f"    f({v}) = {result.vertex_labels[v]}")
    
    print("\n" + "═" * 60)
    print("EDGE LABELS (with verification):")
    for (u, v) in sorted(result.graph.edges()):
        label = result.edge_labels[(u, v)]
        vu = result.vertex_labels[u]
        vv = result.vertex_labels[v]
        total = vu + label + vv
        print(f"  f({u}—{v}) = {label:2d}   [{vu} + {label} + {vv} = {total}]")
else:
    print(f"No EMTL found. Status: {result.status.value}")


## 5. Visualizing Results

Let's create beautiful visualizations of the EMTL.


In [ ]:
# Use the built-in visualizer
if result.exists:
    fig = EMTLVisualizer.visualize(result, figsize=(14, 10), show=False)
    plt.show()


In [ ]:
# Let's try a larger example - Symmetric graph
result_large = solve_emtl(
    m=3, n=3, k=3, t=3,
    timeout=60,
    visualize=False,
    verbose=True
)

if result_large.exists:
    fig = EMTLVisualizer.visualize(result_large, figsize=(16, 10), show=False)
    plt.show()


## 6. Exploring Parameters

Let's systematically explore how different parameters affect the EMTL.


In [ ]:
# Explore varying regularity t

print("Effect of Regularity Parameter t")
print("═" * 50)
print(f"{'t':>3} {'|V|':>5} {'|E|':>5} {'Status':>12} {'Magic k':>10}")
print("─" * 50)

m, n, k = 2, 4, 2

for t in range(n + 1):  # t can be 0 to n
    result = solve_emtl(m=m, n=n, k=k, t=t, 
                       visualize=False, verbose=False, timeout=30)
    
    v_count = m + 2*n + k
    e_count = m*n + n*t + n*k
    status = "Found" if result.exists else result.status.value
    magic = str(result.magic_constant) if result.exists else "—"
    
    print(f"{t:>3} {v_count:>5} {e_count:>5} {status:>12} {magic:>10}")


In [ ]:
# Comprehensive parameter exploration

print("Comprehensive Parameter Study")
print("═" * 70)
print(f"{'Parameters':<15} {'|V|':>5} {'|E|':>5} {'Labels':>7} {'Status':>12} {'Magic k':>10}")
print("─" * 70)

test_cases = [
    (1, 1, 1, 1, "Minimal"),
    (2, 2, 2, 1, "Small"),
    (2, 2, 2, 2, "Small full"),
    (2, 3, 2, 2, "Medium"),
    (3, 3, 3, 3, "Symmetric"),
    (2, 4, 2, 0, "No B-C edges"),
    (4, 4, 4, 4, "Large"),
]

for m, n, k, t, desc in test_cases:
    result = solve_emtl(m=m, n=n, k=k, t=t,
                       visualize=False, verbose=False, timeout=30)
    
    params = GraphParameters(m=m, n=n, k=k, t=t)
    param_str = f"({m},{n},{k},{t})"
    status = "✓ Found" if result.exists else f"✗ {result.status.value}"
    magic = str(result.magic_constant) if result.exists else "—"
    
    print(f"{param_str:<15} {params.num_vertices:>5} {params.num_edges:>5} "
          f"{params.total_labels:>7} {status:>12} {magic:>10}")


## 7. Exercises

Try these exercises to deepen your understanding!

### Exercise 1
Find the EMTL for G(1, 3, 1, 2) and visualize it.

### Exercise 2
What is the largest symmetric graph G(n,n,n,n) for which you can find an EMTL within 30 seconds?

### Exercise 3
Explore what happens when t=0 (graph becomes disconnected). Does EMTL still exist?


In [ ]:
# Exercise 1: Your code here
# Hint: Use solve_emtl(m=1, n=3, k=1, t=2, visualize=True)




In [ ]:
# Exercise 2: Your code here
# Try increasing values of n and measure time




In [ ]:
# Exercise 3: Your code here
# Try different values with t=0




---

## Summary

In this tutorial, we learned:

1. **What is EMTL**: A bijective labeling where all edge sums are equal
2. **Graph Structure**: Four-partition graphs with specific bipartite subgraphs
3. **Using the Solver**: The `solve_emtl` function and result analysis
4. **Visualization**: Creating publication-quality graphs
5. **Parameter Effects**: How m, n, k, t affect EMTL existence

### Next Steps

- Try the web interface: `streamlit run web/app.py`
- Run comprehensive examples: `python examples/run_examples.py`
- Run tests: `pytest tests/ -v`
- Read the full documentation in `README.md`

---

*Happy Labeling!* 🔢
